In [ ]:
import urllib.request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json
import gzip
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Download necessary resources for preprocessing and sentiment analysis
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

In [1]:
dataset_url = "http://snap.stanford.edu/data/amazon/productGraph/kcore_5.json.gz"
save_path = "kcore_5.json.gz"

# Download the 5-core dataset
urllib.request.urlretrieve(dataset_url, save_path)

print("Dataset downloaded successfully.")

In [ ]:
data_file = "reviews_Books_5.json.gz"

# Load the dataset from the JSON file
data = []
with gzip.open(data_file, "rt") as f:
    for line in f:
        entry = json.loads(line)
        data.append(entry)

In [ ]:
# Preprocessing steps
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize the text into individual words
    words = nltk.word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Perform stemming
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # Join the words back into a preprocessed text
    preprocessed_text = ' '.join(words)
    
    return preprocessed_text

In [ ]:
# Preprocess the review texts
preprocessed_review1 = preprocess_text(review1)
preprocessed_review2 = preprocess_text(review2)

# Create a TF-IDF vectorizer object
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the preprocessed review texts
tfidf_matrix = vectorizer.fit_transform([preprocessed_review1, preprocessed_review2])

# Sentiment analysis using VADER SentimentIntensityAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Calculate sentiment scores for the original review texts
sentiment_scores1 = analyzer.polarity_scores(review1)
sentiment_scores2 = analyzer.polarity_scores(review2)

# Accessing the compound sentiment scores
sentiment_score1 = sentiment_scores1['compound']
sentiment_score2 = sentiment_scores2['compound']

print("Sentiment score for review 1:", sentiment_score1)
print("Sentiment score for review 2:", sentiment_score2)